In [268]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
sns.set(font_scale=1, rc={'figure.figsize': (20, 15), "figure.dpi": 100, "savefig.dpi": 100})
sns.set_style("darkgrid")

def warn(*args, **kwargs):
    pass

warnings.warn = warn

# Data Summary and Relationship

This section presents the data and the relationship amongst features. I won't go into much details about the individual time series' since that is not relevant for feature selection.

In [269]:
data = pd.read_csv('data/dataset.csv')
data['date'] = pd.to_datetime(data['date'], format="%Y-%m-%d")
data.set_index('date', inplace=True)

A summary of data characteristics if provided below.

In [270]:
stats = data.describe()
stats.loc['skew'] = data.skew().tolist()
stats.loc['kurt'] = data.kurtosis().tolist()
stats

price        demand  generation_coal  generation_diesel  \
count  3226.000000   3226.000000      3226.000000        3226.000000   
mean    100.569646  20980.063938      3818.922010          12.598392   
std      70.509956   1862.658455      3664.952344          80.918839   
min       0.016808  15468.521200         0.000000           0.000000   
25%      55.517461  19698.108575         0.000000           0.000000   
50%      79.132532  20999.210400      3465.956500           0.000000   
75%     123.057582  22160.136650      5481.356250           0.000000   
max    1216.117213  27022.325700     17213.768000        1895.695000   
skew      2.888983      0.065995         0.988345          11.141217   
kurt     23.025562     -0.213533         0.765612         163.854196   

       generation_gas  generation_geo  generation_hydro  generation_wind  \
count     3226.000000     3226.000000       3226.000000      3226.000000   
mean     15193.888944    19944.658071      66339.230811      5321.149008   
std       5007.137430     1462.082847       8705.602119      2717.717648   
min       1753.844000    13436.056323      38516.216832       186.309891   
25%      11765.739975    19365.412617      60445.832117      3153.795119   
50%      14755.118530    20317.658681      66485.942745      4994.229307   
75%      18566.258120    20914.497774      72400.887920      7357.672952   
max      30956.428640    22967.320000      93199.082000     14736.434000   
skew         0.210042       -1.233601         -0.079656         0.427651   
kurt        -0.026651        1.659903         -0.161182        -0.386208   

       generation_wood   cloudcover          dew     humidity       precip  \
count      3226.000000  3226.000000  3226.000000  3226.000000  3226.000000   
mean        681.485585    53.922357     9.190771    79.972138     3.099663   
std         246.182035    16.164405     3.308243     5.548270     3.638905   
min           0.000000     9.053024     0.092167    62.852095     0.000000   
25%         661.006500    41.683506     6.759679    76.192119     0.671309   
50%         778.870000    53.578988     9.032762    80.084393     1.910983   
75%         834.107500    65.623655    11.661095    83.975798     4.126609   
max         931.137000    97.552333    18.514595    94.739143    36.750307   
skew         -1.723990     0.067310     0.046727    -0.157033     2.665342   
kurt          1.889052    -0.595559    -0.612546    -0.384720    11.308852   

       precipcover  solarenergy         temp      winddir    windspeed  
count  3226.000000  3226.000000  3226.000000  3226.000000  3226.000000  
mean      8.929866    15.063008    12.865396   201.107166    28.433654  
std       5.516833     7.754738     3.577091    59.262319     6.743033  
min       0.000000     1.308381     4.167548    34.453952    12.451976  
25%       4.637363     7.955679     9.932024   157.785024    23.506482  
50%       8.410038    14.167190    12.690119   205.337369    27.599750  
75%      12.396362    21.469732    15.813262   245.807357    32.733911  
max      47.231779    33.975381    21.726524   334.999500    56.459238  
skew      0.831280     0.333476     0.071250    -0.239384     0.589065  
kurt      1.867591    -1.074538    -0.916966    -0.631905     0.187883

We want to identify the relationship between variables and so let's visualize the correlation matrix.

In [271]:
sns.heatmap(data.corr(), annot=True);

Some observations from the correlation heatmap:

- The generation of **coal** and **diesel** exhibit a strong positive relationship with **price**. Non-renewable fuel in New Zealand is largely used during peak hours, and to a greater extent when there is a hydro shortage. Running coal and diesel generators is significantly more expensive than renewable energy so this relationship is not surprising.
- **Price** and **demand** are positively correlated. Electricity demand is largely inelastic with respect to most external factors, whereas price is prone to significant deviation with changes in external factors, irrespective of demand.
- **Geothermal**, **natural gas** and **hydro** generation are positively correlated with **demand**. They are considered baseline generation sources and are a constant source of electricity. Their generation output throughout the day is ramped up or down depending on demand.
- **Demand** is negatively correlated with **temperature**, **dew point**, and **solar energy**. Winters in New Zealand generally observe higher demand for electricity due to higher usage of heat pumps. Solar energy corresponds to the day-night cycle and demand exhibits an intra-day cycle as well, due to higher industrial and residential usage at different times of the day.
- It is interesting to observe the correlation amongs generation fuels because it explains the role these fuels have in the electricity market. Consider the negative correlation between **coal** and **hydro** generation. As New Zealand is a highly renewable electricity market, its dependence on coal is significant only when there is a shortage of hydro fuel. Coal is largely a backup plan, employed only during peak demand hours or when there is a lack of renewable energy.
- **Hydro** generation does not have a significant relationship with **precipitation**. However, it is important to note that this correlation is an instantaneous measure. Higher precipitation periods would certainly impact future hydro generation due to higher lake levels and inflows.
- It comes as no surprise that **wind generation** is positively correlated with **wind speeds** and **direction**. Unlike hydro fuel, wind is not storable (yet?) and its generation depends on instantaneous wind levels.
- The significant relationships amongst weather variables are self-explanatory.

We do not need to dwell on these relationships much longer. The primary purpose of this project is feature selection using LASSO. However, feature selection itself is motivated as a tool to obtain robust estimates, to reduce dimensionality for easier computation and analysis, and to avoid over-fitting. 

We are working with time-series data and it is only natural to consider the effect that lagged-values have on future data. One example of this was explained above, with current precipitation having no relationship with current hydro generation, but it certainly would with future generation. For this purpose, we create new features in our dataset which are lagged values of the current features. I have chosen an arbitrary value of 30 time lags for each feature.

# Building Feature Set

## Scaling Data

Before creating a feature set of lagged variables, it would be prudent to transform the data so that they have a similar scale. I don't want to use standardization or min-max scaling since many of the features have outliers or extreme ranges. Instead, I will use a power transformation, the Yeo-Johnson method from the ``scikit-learn`` library. The Yeo-Johnson transformation is defined as 

\begin{align*}
y &= ((x + 1)^\lambda - 1) / \lambda \quad \text{for positive data, and} \\
y &= -((-x + 1)^\lambda - 1) / \lambda \quad \text{for negative data.}
\end{align*}

The optimal value for $\lambda$ is chosen through maximum likelihood estimation.

In [272]:
from sklearn.preprocessing import PowerTransformer

In [273]:
pt = PowerTransformer(method='yeo-johnson')
pt = pt.fit(data)
scaled_data = pt.transform(data)

Let's convert the scaled data into a DataFrame object.

In [274]:
scaled_data = DataFrame(scaled_data)
scaled_data.columns = data.columns.values

In [275]:
scaled_data

price    demand  generation_coal  generation_diesel  generation_gas  \
0    -1.935973 -2.699566        -1.492301          -0.331501       -1.405115   
1    -2.199742 -3.075400        -1.492301          -0.331501       -1.354476   
2    -1.585592 -2.390848        -1.492301          -0.331501       -1.334000   
3    -1.511914 -2.223537        -1.492301          -0.331501       -1.345057   
4    -1.555152 -2.568987        -1.492301          -0.331501       -1.408213   
...        ...       ...              ...                ...             ...   
3221  0.162304  0.566377        -0.349778          -0.331501       -1.632599   
3222  0.177994  0.240194        -1.492301          -0.331501       -1.499349   
3223 -1.122500 -0.796184        -1.492301          -0.331501       -1.724213   
3224 -1.078066 -1.021814        -1.492301          -0.331501       -1.844280   
3225 -1.156724 -0.085491        -0.546353          -0.331501       -1.913505   

      generation_geo  generation_hydro  generation_wind  generation_wood  \
0          -1.632013         -1.090615        -0.045802         0.692602   
1          -1.649098         -1.501784         0.890079         0.247940   
2          -1.386262         -0.912121         0.988679         0.489499   
3          -0.996040         -0.806616         0.217833         0.215442   
4          -1.327804         -1.072888         1.161545         0.564996   
...              ...               ...              ...              ...   
3221        0.965688          0.923287         1.151074        -0.396474   
3222        1.191745          0.626042         0.932915         0.314241   
3223        0.913871         -0.901667         2.142412         0.675005   
3224        1.026469         -0.457778         0.466871         0.559958   
3225        0.851882          1.063887        -0.893629         0.409538   

      cloudcover       dew  humidity    precip  precipcover  solarenergy  \
0       0.455447  0.337660 -0.655140  1.133046     2.009602     0.985468   
1      -0.103555  0.418642 -1.238662  0.740146     1.189975     1.385643   
2       0.125156  0.959579 -0.638221  0.964228     0.625469     1.363951   
3      -0.013985  1.065897 -0.580910 -0.729789     0.079755     1.390269   
4      -0.964508  1.156831 -0.320690  1.361457     1.546032     1.293140   
...          ...       ...       ...       ...          ...          ...   
3221    1.096734  0.374680  0.115505  0.056417    -0.986612     0.197686   
3222    1.438221  0.992325  0.704445  1.292826    -0.254784    -0.054687   
3223    1.878619  1.551155  0.839849  1.535014     0.418940    -0.340604   
3224    1.551300  1.307750  0.710769  2.374429     1.227370    -0.375796   
3225    0.332400  1.178389  0.651566  0.446842     0.064495     0.549038   

          temp   winddir  windspeed  
0     0.518210  1.924111   0.005510  
1     0.824979  1.886176   0.637331  
2     1.120501  2.113579   2.052320  
3     1.186518  1.580615   0.743958  
4     1.216529  1.775604   1.941656  
...        ...       ...        ...  
3221  0.317488 -0.145757  -0.789833  
3222  0.727838 -0.572586  -0.086211  
3223  1.208359 -0.063732   0.400041  
3224  1.063146  0.157725   0.238021  
3225  0.911993 -0.062096  -1.070157  

[3226 rows x 18 columns]

## Building Lagged Features

The function below will generate the specified number of lagged features for the input data.

In [276]:
def build_lags(data, n_lags=1):
    df = DataFrame(data)
    cols, names = list(), list()
    col_names = list(data)

    for i in range(n_lags, 0, -1):
        cols.append(df.shift(i))
        names += [('%s (t-%d)' % (j, i)) for j in col_names]
        
    cols.append(df.shift(-i))
    names += [('%s (t)' % (j)) for j in col_names]

    agg = concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    
    return agg

In [277]:
num_lags = 30
lag_data = build_lags(scaled_data, n_lags=num_lags)

In [278]:
lag_data

price (t-30)  demand (t-30)  generation_coal (t-30)  \
30       -1.935973      -2.699566               -1.492301   
31       -2.199742      -3.075400               -1.492301   
32       -1.585592      -2.390848               -1.492301   
33       -1.511914      -2.223537               -1.492301   
34       -1.555152      -2.568987               -1.492301   
...            ...            ...                     ...   
3220     -1.475493      -0.287565                0.432956   
3221     -0.701551       0.574425               -0.669224   
3222     -0.786450       0.617746               -1.492301   
3223     -1.077248       0.725256               -1.492301   
3224     -0.866711       1.035833                0.451295   

      generation_diesel (t-30)  generation_gas (t-30)  generation_geo (t-30)  \
30                   -0.331501              -1.405115              -1.632013   
31                   -0.331501              -1.354476              -1.649098   
32                   -0.331501              -1.334000              -1.386262   
33                   -0.331501              -1.345057              -0.996040   
34                   -0.331501              -1.408213              -1.327804   
...                        ...                    ...                    ...   
3220                 -0.331501              -2.841415               1.360141   
3221                 -0.331501              -1.245795               0.044232   
3222                 -0.331501              -1.257842              -0.240668   
3223                 -0.331501              -1.429012               0.903717   
3224                 -0.331501              -3.008181               1.005733   

      generation_hydro (t-30)  generation_wind (t-30)  generation_wood (t-30)  \
30                  -1.090615               -0.045802                0.692602   
31                  -1.501784                0.890079                0.247940   
32                  -0.912121                0.988679                0.489499   
33                  -0.806616                0.217833                0.215442   
34                  -1.072888                1.161545                0.564996   
...                       ...                     ...                     ...   
3220                 0.356515                0.989228               -0.035716   
3221                 1.685796               -0.901573                0.168547   
3222                 1.479869               -0.059724                0.382001   
3223                 0.931206                1.313067                0.203727   
3224                 1.312516                1.991742                0.219525   

      cloudcover (t-30)  ...  generation_wood (t)  cloudcover (t)   dew (t)  \
30             0.455447  ...             0.944739       -0.887582  0.714194   
31            -0.103555  ...             0.842632       -1.746522  0.480167   
32             0.125156  ...             0.851931       -2.546542  0.738029   
33            -0.013985  ...             0.397324       -1.428696  0.948421   
34            -0.964508  ...             0.315020       -0.187213  0.733743   
...                 ...  ...                  ...             ...       ...   
3220           1.831061  ...            -0.396474        1.096734  0.374680   
3221           1.312947  ...             0.314241        1.438221  0.992325   
3222           1.665379  ...             0.675005        1.878619  1.551155   
3223           2.095924  ...             0.559958        1.551300  1.307750   
3224           1.870950  ...             0.409538        0.332400  1.178389   

      humidity (t)  precip (t)  precipcover (t)  solarenergy (t)  temp (t)  \
30       -0.746612   -0.853114        -0.653830         1.240020  0.905486   
31       -1.414070   -1.537971        -1.952905         1.522047  0.952371   
32       -1.232479   -1.749997        -2.298959         1.616088  1.143326   
33       -0.770893   -1.449337        -1.464085         1.539882  1.152467   


Perfect! Now I will specify my target vector and feature set. 

The target variable is `price (t)`.

For features, we want to use all variables but only as recent as time `t-1`. I don't want to include variables at time `t`. In practice, this would really depend on the order in which data is obtained in real time. But I'm keeping it simple.

In [279]:
y = lag_data[['price (t)']]
y.shape

(3195, 1)

In [280]:
X = lag_data.loc[:, :'windspeed (t-1)']
X.shape

(3195, 540)

So we our feature set contains 540 features. Now, we can use LASSO to perform variable selection on our feature set.

# Feature Selection using LASSO

The LASSO (least absolute shrinkage and selection operator) is a type of regularization method that is used to improve the predictive performance of linear regression models. It works by adding a penalty term to the objective function that is being optimized. The penalty term is defined as the sum of the absolute values of the model coefficients, multiplied by a hyperparameter called the regularization parameter, denoted $\alpha$. 

The regularization strength determines how much the coefficients of the model should be penalized. When the regularization strength is set to a high value, the coefficients of the model will be strongly penalized, which will result in a model with fewer non-zero coefficients (i.e., a model with fewer features). When the regularization strength is set to a low value, the coefficients of the model will be less heavily penalized, which will result in a model with more non-zero coefficients (i.e., a model with more features).

The goal of LASSO regularization is to balance the trade-off between model complexity and model performance. By penalizing the coefficients of the model, LASSO can help to prevent overfitting and improve the generalization performance of the model. In this way, it serves as a feature selection technique.

Firstly, I will build the cross-validation sets for LASSO estimation. It's important to remember that the features are time-series' and cross-validation must preserve the sequential nature of our data. The `scikit-learn` library has a module just for this.

In [281]:
from sklearn.model_selection import TimeSeriesSplit

I want a 50-fold cross-validation scheme. This seems like a large number, but my idea is to assess how many times each feature appears is chosen (not reduced to 0) out of the 50 cross-validation sets.

In [282]:
ts_cv = TimeSeriesSplit(n_splits=50, gap=0)
splits = list(ts_cv.split(X, y))

## Evaluate LASSO with Cross-Validation

In [283]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from numpy import arange

I use a grid search algorithm to identify the optimal regularization parameter $\alpha$ for LASSO. I only search for $\alpha$ values between 0 and 0.1, since I already know the optimal solution lies in this range. In practice, it may be necessary to increase the upper bound of the search range. I use the mean absolute error as the scoring criteria.

In [289]:
model = Lasso()
grid = dict()
grid['alpha'] = arange(0, 0.1, 0.001)
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=ts_cv, verbose=3)
results = search.fit(X, y)

Fitting 50 folds for each of 100 candidates, totalling 5000 fits
[CV 1/50] END .......................alpha=0.0;, score=-1.021 total time=   0.0s
[CV 2/50] END .......................alpha=0.0;, score=-0.598 total time=   0.0s
[CV 3/50] END .......................alpha=0.0;, score=-0.656 total time=   0.0s
[CV 4/50] END .......................alpha=0.0;, score=-0.679 total time=   0.0s
[CV 5/50] END .......................alpha=0.0;, score=-0.771 total time=   0.0s
[CV 6/50] END .......................alpha=0.0;, score=-1.043 total time=   0.1s
[CV 7/50] END .......................alpha=0.0;, score=-0.952 total time=   0.1s
[CV 8/50] END .......................alpha=0.0;, score=-1.069 total time=   0.1s
[CV 9/50] END .......................alpha=0.0;, score=-0.603 total time=   0.2s
[CV 10/50] END ......................alpha=0.0;, score=-0.505 total time=   0.1s
[CV 11/50] END ......................alpha=0.0;, score=-0.608 total time=   0.2s
[CV 12/50] END ......................alpha=0

In [303]:
grid['alpha'].tolist()

[0.0,
 0.001,
 0.002,
 0.003,
 0.004,
 0.005,
 0.006,
 0.007,
 0.008,
 0.009000000000000001,
 0.01,
 0.011,
 0.012,
 0.013000000000000001,
 0.014,
 0.015,
 0.016,
 0.017,
 0.018000000000000002,
 0.019,
 0.02,
 0.021,
 0.022,
 0.023,
 0.024,
 0.025,
 0.026000000000000002,
 0.027,
 0.028,
 0.029,
 0.03,
 0.031,
 0.032,
 0.033,
 0.034,
 0.035,
 0.036000000000000004,
 0.037,
 0.038,
 0.039,
 0.04,
 0.041,
 0.042,
 0.043000000000000003,
 0.044,
 0.045,
 0.046,
 0.047,
 0.048,
 0.049,
 0.05,
 0.051000000000000004,
 0.052000000000000005,
 0.053,
 0.054,
 0.055,
 0.056,
 0.057,
 0.058,
 0.059000000000000004,
 0.06,
 0.061,
 0.062,
 0.063,
 0.064,
 0.065,
 0.066,
 0.067,
 0.068,
 0.069,
 0.07,
 0.07100000000000001,
 0.07200000000000001,
 0.073,
 0.074,
 0.075,
 0.076,
 0.077,
 0.078,
 0.079,
 0.08,
 0.081,
 0.082,
 0.083,
 0.084,
 0.085,
 0.08600000000000001,
 0.08700000000000001,
 0.088,
 0.089,
 0.09,
 0.091,
 0.092,
 0.093,
 0.094,
 0.095,
 0.096,
 0.097,
 0.098,
 0.099]

In [309]:
-np.array(results.cv_results_['mean_test_score'].tolist())

array([0.52956069, 0.45188957, 0.43604363, 0.42391351, 0.41652305,
       0.41099909, 0.40752164, 0.40513107, 0.40340685, 0.4023428 ,
       0.40169161, 0.40141383, 0.40125958, 0.40118469, 0.40109368,
       0.40112847, 0.40124266, 0.40136176, 0.40156293, 0.40188704,
       0.40221384, 0.40253639, 0.4029069 , 0.40332075, 0.40384852,
       0.40449173, 0.40518881, 0.40591395, 0.40665107, 0.40740319,
       0.40815297, 0.4089069 , 0.40966953, 0.41042893, 0.4110615 ,
       0.41174136, 0.41244233, 0.4131851 , 0.41396383, 0.41476211,
       0.41558074, 0.41640411, 0.41725447, 0.41812208, 0.41898994,
       0.41987964, 0.42078548, 0.42171643, 0.42265632, 0.42361157,
       0.42456767, 0.42559105, 0.42666844, 0.42771869, 0.42876396,
       0.42979962, 0.43079724, 0.43179019, 0.43277077, 0.43375067,
       0.4347294 , 0.43572201, 0.43669979, 0.43768088, 0.43867609,
       0.43968329, 0.44067351, 0.44166023, 0.44264715, 0.4436375 ,
       0.44462516, 0.44561733, 0.44660511, 0.4475957 , 0.44857

In [310]:
plt.plot(grid['alpha'].tolist(), -np.array(results.cv_results_['mean_test_score'].tolist()))